In [53]:
import pandas as pd
import requests
import datetime 

In [13]:
sites_response = requests.get("http://api.erg.kcl.ac.uk/AirQuality/Information/MonitoringSites/GroupName=London/Json")

In [39]:
unfiltered_sites = pd.DataFrame(sites_response.json()['Sites']['Site'])
unfiltered_sites.head()

,@LocalAuthorityCode,@LocalAuthorityName,@SiteCode,@SiteName,@SiteType,@DateClosed,@DateOpened,@Latitude,@Longitude,@LatitudeWGS84,@LongitudeWGS84,@DisplayOffsetX,@DisplayOffsetY,@DataOwner,@DataManager,@SiteLink
0,27,Richmond,TD0,"- National Physical Laboratory, Teddington",Suburban,2018-01-01 00:00:00,1996-08-08 00:00:00,51.4243043441456,-0.345714576446947,6696103.27675,-37808.8858115,0,-200,Richmond,King's College London,http://www.londonair.org.uk/london/asp/publicd...
1,1,Barking and Dagenham,BG3,Barking and Dagenham - North Street,Kerbside,2011-05-25 00:00:00,2007-03-16 00:00:00,51.540444,0.074418,6717454.5833,8284.17386585,0,0,Barking and Dagenham,King's College London,http://www.londonair.org.uk/london/asp/publicd...
2,1,Barking and Dagenham,BG1,Barking and Dagenham - Rush Green,Suburban,,1999-11-02 00:00:00,51.563752,0.177891,6721627.34498,19802.7355367,0,0,Barking and Dagenham,King's College London,http://www.londonair.org.uk/london/asp/publicd...
3,1,Barking and Dagenham,BG2,Barking and Dagenham - Scrattons Farm,Suburban,,1999-10-17 00:00:00,51.529389,0.132857,6715476.18683,14789.5735883,0,0,Barking and Dagenham,King's College London,http://www.londonair.org.uk/london/asp/publicd...
4,2,Barnet,BN2,Barnet - Finchley,Urban Background,2012-04-20 00:00:00,2000-08-09 13:00:00,51.591901,-0.205992,6726669.62886,-22930.9245475,0,0,Barnet,King's College London,http://www.londonair.org.uk/london/asp/publicd...


In [62]:
sites = unfiltered_sites.filter(["@LocalAuthorityName", "@SiteCode", "@SiteName", "@SiteType", "@DateClosed", "@DateOpened", "@Latitude", "@Longitude"])
sites = sites.rename(columns={"@LocalAuthorityName":"BoroughName", "@SiteCode":"SiteCode", "@SiteName":"SiteName", "@SiteType":"SiteType", "@DateClosed":"DateClosed", "@DateOpened":"DateOpened", "@Latitude":"Latitude", "@Longitude":"Longitude"})
sites.head()

,BoroughName,SiteCode,SiteName,SiteType,DateClosed,DateOpened,Latitude,Longitude
0,Richmond,TD0,"- National Physical Laboratory, Teddington",Suburban,2018-01-01 00:00:00,1996-08-08 00:00:00,51.4243043441456,-0.345714576446947
1,Barking and Dagenham,BG3,Barking and Dagenham - North Street,Kerbside,2011-05-25 00:00:00,2007-03-16 00:00:00,51.540444,0.074418
2,Barking and Dagenham,BG1,Barking and Dagenham - Rush Green,Suburban,,1999-11-02 00:00:00,51.563752,0.177891
3,Barking and Dagenham,BG2,Barking and Dagenham - Scrattons Farm,Suburban,,1999-10-17 00:00:00,51.529389,0.132857
4,Barnet,BN2,Barnet - Finchley,Urban Background,2012-04-20 00:00:00,2000-08-09 13:00:00,51.591901,-0.205992


In [74]:
sites['DateClosed'] =  pd.to_datetime(sites['DateClosed'], format='%Y-%m-%d %H:%M:%S')
sites['DateOpened'] =  pd.to_datetime(sites['DateOpened'], format='%Y-%m-%d %H:%M:%S')

In [81]:
date_mask = ((pd.isnull(sites['DateClosed'])) | (sites['DateClosed'] > datetime.datetime(2012, 12, 31))) & (sites['DateOpened'] < datetime.datetime(2012, 1, 1))
filtered_sites = sites[date_mask]
filtered_sites

,BoroughName,SiteCode,SiteName,SiteType,DateClosed,DateOpened,Latitude,Longitude
0,Richmond,TD0,"- National Physical Laboratory, Teddington",Suburban,2018-01-01,1996-08-08 00:00:00,51.4243043441456,-0.345714576446947
2,Barking and Dagenham,BG1,Barking and Dagenham - Rush Green,Suburban,NaT,1999-11-02 00:00:00,51.563752,0.177891
3,Barking and Dagenham,BG2,Barking and Dagenham - Scrattons Farm,Suburban,NaT,1999-10-17 00:00:00,51.529389,0.132857
8,Bexley,BX2,Bexley - Belvedere,Suburban,NaT,1998-01-24 15:00:00,51.4906102082147,0.158914493927518
9,Bexley,BX0,Bexley - Belvedere FDMS,Suburban,NaT,2004-04-28 00:00:00,51.4906102082147,0.158914493927518
...,...,...,...,...,...,...,...,...
218,Westminster,WM5,Westminster - Covent Garden,Urban Background,NaT,2009-07-10 00:00:00,51.5119769825798,-0.12162720332856
221,Westminster,WM0,Westminster - Horseferry Road,Urban Background,NaT,2001-07-17 00:00:00,51.494681,-0.131938
222,Westminster,MY1,Westminster - Marylebone Road,Kerbside,NaT,1997-05-26 12:00:00,51.52254,-0.15459
223,Westminster,MY7,Westminster - Marylebone Road FDMS,Kerbside,NaT,2004-01-22 12:00:00,51.52254,-0.15459


In [150]:
latDict = pd.Series(filtered_sites['Latitude'].values ,index=filtered_sites['SiteCode']).to_dict()
lonDict = pd.Series(filtered_sites['Longitude'].values ,index=filtered_sites['SiteCode']).to_dict()
latDict, lonDict

({'TD0': '51.4243043441456',
  'BG1': '51.563752',
  'BG2': '51.529389',
  'BX2': '51.4906102082147',
  'BX0': '51.4906102082147',
  'BQ7': '51.4946486813055',
  'BQ8': '51.4946486813055',
  'BX4': '51.477670796298',
  'BQ6': '51.477670796298',
  'BQ5': '51.4781935083933',
  'BX1': '51.4659832746662',
  'BX9': '51.4659832746662',
  'BX3': '51.510808',
  'BT4': '51.552476',
  'BT6': '51.537799',
  'BT5': '51.552656',
  'BY7': '51.405546',
  'BL0': '51.522287',
  'KX4': '51.533238',
  'CD9': '51.527975',
  'CD3': '51.515532',
  'CD1': '51.544219',
  'CT4': '51.520225273171',
  'CT2': '51.5145253362314',
  'CT1': '51.511491',
  'CT3': '51.513847178423',
  'CT8': '51.5095436853913',
  'CT6': '51.510499',
  'CR4': '51.374312',
  'CR5': '51.411349',
  'CR3': '51.40401',
  'EA6': '51.53085',
  'EA8': '51.518948',
  'EI8': '51.5189160348806',
  'EA7': '51.508549',
  'EI7': '51.5085164816243',
  'EI2': '51.5058699783726',
  'EI1': '51.5236078191481',
  'EN5': '51.613865',
  'EN1': '51.645036726

In [88]:
site_codes = filtered_sites['SiteCode']
species_codes = ["CO", "NO2", "O3", "PM10", "PM25", "SO2"]

In [142]:
def get_species_data(species):
#     species_data = pd.DataFrame({'Site':[], 'Latitude':[], 'Longitude':[], 'Species':[], 'Date':[], 'AverageValue':[]})
    frames = []
    for site in site_codes:
        url = f"http://api.erg.kcl.ac.uk/AirQuality/Data/SiteSpecies/SiteCode={site}/SpeciesCode={species}/StartDate=2012-05-01/EndDate=2012-10-31/Json"
        response = requests.get(url)
        df = pd.DataFrame(response.json()['RawAQData']['Data'])
        df = df.rename(columns={"@MeasurementDateGMT":"Date", "@Value":"AverageValue"})
        df['Date'] =  pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
        df['AverageValue'] = pd.to_numeric(df['AverageValue'],errors='coerce')
        df = df.resample('D', on='Date').mean()
        df['Site'] = site
#         df['Latitude'] = filtered_sites.loc[filtered_sites['SiteCode'] == site]['Latitude'][0]
#         df['Longitude'] = filtered_sites.loc[filtered_sites['SiteCode'] == site]['Longitude'][0]
        df['Species'] = species
        frames.append(df)
    result = pd.concat(frames)
    return result
        

In [90]:
url = f"http://api.erg.kcl.ac.uk/AirQuality/Data/SiteSpecies/SiteCode=TD0/SpeciesCode=NO2/StartDate=2012-01-01/EndDate=2012-12-31/Json"
response = requests.get(url)
response.json()

{'RawAQData': {'@SiteCode': 'TD0',
  '@SpeciesCode': 'NO2',
  'Data': [{'@MeasurementDateGMT': '2012-01-01 00:00:00', '@Value': '4.7'},
   {'@MeasurementDateGMT': '2012-01-01 01:00:00', '@Value': ''},
   {'@MeasurementDateGMT': '2012-01-01 02:00:00', '@Value': '5.5'},
   {'@MeasurementDateGMT': '2012-01-01 03:00:00', '@Value': '3.4'},
   {'@MeasurementDateGMT': '2012-01-01 04:00:00', '@Value': '3.4'},
   {'@MeasurementDateGMT': '2012-01-01 05:00:00', '@Value': '3.5'},
   {'@MeasurementDateGMT': '2012-01-01 06:00:00', '@Value': '4'},
   {'@MeasurementDateGMT': '2012-01-01 07:00:00', '@Value': '4'},
   {'@MeasurementDateGMT': '2012-01-01 08:00:00', '@Value': '3.9'},
   {'@MeasurementDateGMT': '2012-01-01 09:00:00', '@Value': '5.4'},
   {'@MeasurementDateGMT': '2012-01-01 10:00:00', '@Value': '4.8'},
   {'@MeasurementDateGMT': '2012-01-01 11:00:00', '@Value': '5.3'},
   {'@MeasurementDateGMT': '2012-01-01 12:00:00', '@Value': '6.8'},
   {'@MeasurementDateGMT': '2012-01-01 13:00:00', '@Val

In [98]:
df = pd.DataFrame(response.json()['RawAQData']['Data'])

In [117]:
species = "NO2"
df = df.rename(columns={"@MeasurementDateGMT":"Date", "@Value":f"{species} Value"})
df

,Date,NO2 Value
0,2012-01-01 00:00:00,4.7
1,2012-01-01 01:00:00,NaN
2,2012-01-01 02:00:00,5.5
3,2012-01-01 03:00:00,3.4
4,2012-01-01 04:00:00,3.4
...,...,...
8755,2012-12-30 19:00:00,30.7
8756,2012-12-30 20:00:00,29.5
8757,2012-12-30 21:00:00,25.8
8758,2012-12-30 22:00:00,25.3


In [118]:
df['Date'] =  pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')

In [119]:
df['NO2 Value'] = pd.to_numeric(df['NO2 Value'],errors='coerce')

In [125]:
df1 = df.resample('D', on='Date').mean()
df1.sort_values(by="NO2 Value")

,NO2 Value
Date,
2012-01-03,6.895833
2012-01-01,7.273913
2012-01-21,8.104167
2012-01-02,8.404167
2012-01-04,10.612500
...,...
2012-10-18,NaN
2012-10-19,NaN
2012-10-20,NaN


In [130]:
df1['Site'] = "TD0"

In [131]:
df1

,NO2 Value,Site
Date,,
2012-01-01,7.273913,TD0
2012-01-02,8.404167,TD0
2012-01-03,6.895833,TD0
2012-01-04,10.612500,TD0
2012-01-05,15.150000,TD0
...,...,...
2012-12-26,33.283333,TD0
2012-12-27,30.695833,TD0
2012-12-28,34.833333,TD0


In [141]:
! python --version

Python 3.7.4


In [143]:
CO_data = get_species_data("CO")
CO_data

,AverageValue,Site,Species
Date,,,
2012-05-01,NaN,TD0,CO
2012-05-02,NaN,TD0,CO
2012-05-03,NaN,TD0,CO
2012-05-04,NaN,TD0,CO
2012-05-05,NaN,TD0,CO
...,...,...,...
2012-10-26,NaN,WM6,CO
2012-10-27,NaN,WM6,CO
2012-10-28,NaN,WM6,CO


In [151]:
CO_data_copy = CO_data

In [153]:
CO_data_copy['Latitude'] = CO_data_copy['Site'].map(latDict)
CO_data_copy

,AverageValue,Site,Species,Latitude
Date,,,,
2012-05-01,NaN,TD0,CO,51.4243043441456
2012-05-02,NaN,TD0,CO,51.4243043441456
2012-05-03,NaN,TD0,CO,51.4243043441456
2012-05-04,NaN,TD0,CO,51.4243043441456
2012-05-05,NaN,TD0,CO,51.4243043441456
...,...,...,...,...
2012-10-26,NaN,WM6,CO,51.5139287404213
2012-10-27,NaN,WM6,CO,51.5139287404213
2012-10-28,NaN,WM6,CO,51.5139287404213


In [154]:
CO_data_copy['Longitude'] = CO_data_copy['Site'].map(lonDict)
CO_data_copy

,AverageValue,Site,Species,Latitude,Longitude
Date,,,,,
2012-05-01,NaN,TD0,CO,51.4243043441456,-0.345714576446947
2012-05-02,NaN,TD0,CO,51.4243043441456,-0.345714576446947
2012-05-03,NaN,TD0,CO,51.4243043441456,-0.345714576446947
2012-05-04,NaN,TD0,CO,51.4243043441456,-0.345714576446947
2012-05-05,NaN,TD0,CO,51.4243043441456,-0.345714576446947
...,...,...,...,...,...
2012-10-26,NaN,WM6,CO,51.5139287404213,-0.152792701881935
2012-10-27,NaN,WM6,CO,51.5139287404213,-0.152792701881935
2012-10-28,NaN,WM6,CO,51.5139287404213,-0.152792701881935


In [155]:
CO_data_copy = CO_data_copy[['Site', 'Latitude', 'Longitude', 'Species', 'Date', 'AverageValue']]
CO_data_copy

KeyError: "['Date'] not in index"

In [158]:
CO_data_copy.to_csv('CO-data.csv')

In [159]:
NO2_data = get_species_data("NO2")
NO2_data

,AverageValue,Site,Species
Date,,,
2012-05-01,24.775000,TD0,NO2
2012-05-02,24.633333,TD0,NO2
2012-05-03,32.216667,TD0,NO2
2012-05-04,33.066667,TD0,NO2
2012-05-05,21.504167,TD0,NO2
...,...,...,...
2012-10-26,NaN,WM6,NO2
2012-10-27,NaN,WM6,NO2
2012-10-28,NaN,WM6,NO2


In [163]:
NO2_data

,AverageValue,Site,Species
Date,,,
2012-05-01,24.775000,TD0,NO2
2012-05-02,24.633333,TD0,NO2
2012-05-03,32.216667,TD0,NO2
2012-05-04,33.066667,TD0,NO2
2012-05-05,21.504167,TD0,NO2
...,...,...,...
2012-10-26,NaN,WM6,NO2
2012-10-27,NaN,WM6,NO2
2012-10-28,NaN,WM6,NO2


In [162]:
def clean_and_export(df, species):
    df['Latitude'] = df['Site'].map(latDict)
    df['Longitude'] = df['Site'].map(lonDict)
    df.to_csv(species + '-data.csv')

In [164]:
clean_and_export(NO2_data, "NO2")

In [165]:
O3_data = get_species_data("O3")
O3_data

,AverageValue,Site,Species
Date,,,
2012-05-01,53.695833,TD0,O3
2012-05-02,45.675000,TD0,O3
2012-05-03,44.795833,TD0,O3
2012-05-04,46.300000,TD0,O3
2012-05-05,60.604167,TD0,O3
...,...,...,...
2012-10-26,NaN,WM6,O3
2012-10-27,NaN,WM6,O3
2012-10-28,NaN,WM6,O3


In [166]:
clean_and_export(O3_data, "O3")

In [168]:
PM10_data = get_species_data("PM10")
PM10_data

,AverageValue,Site,Species
Date,,,
2012-05-01,NaN,TD0,PM10
2012-05-02,NaN,TD0,PM10
2012-05-03,NaN,TD0,PM10
2012-05-04,NaN,TD0,PM10
2012-05-05,NaN,TD0,PM10
...,...,...,...
2012-10-26,NaN,WM6,PM10
2012-10-27,NaN,WM6,PM10
2012-10-28,NaN,WM6,PM10


In [169]:
clean_and_export(PM10_data, "PM10")

In [170]:
PM25_data = get_species_data("PM25")
PM25_data

,AverageValue,Site,Species
Date,,,
2012-05-01,11.875000,TD0,PM25
2012-05-02,19.125000,TD0,PM25
2012-05-03,8.041667,TD0,PM25
2012-05-04,9.208333,TD0,PM25
2012-05-05,4.000000,TD0,PM25
...,...,...,...
2012-10-26,NaN,WM6,PM25
2012-10-27,NaN,WM6,PM25
2012-10-28,NaN,WM6,PM25


In [171]:
clean_and_export(PM25_data, "PM25")

In [172]:
SO2_data = get_species_data("SO2")
SO2_data

,AverageValue,Site,Species
Date,,,
2012-05-01,NaN,TD0,SO2
2012-05-02,NaN,TD0,SO2
2012-05-03,NaN,TD0,SO2
2012-05-04,NaN,TD0,SO2
2012-05-05,NaN,TD0,SO2
...,...,...,...
2012-10-26,NaN,WM6,SO2
2012-10-27,NaN,WM6,SO2
2012-10-28,NaN,WM6,SO2


In [173]:
clean_and_export(SO2_data, "SO2")